# Pivot Translation with M2M100

**Pivot translation** is a technique used in **low-resource language translation**, where the direct translation between two languages may not perform well due to lack of data. Instead, translation is performed in two stages:

* **Source → Pivot (High-resource Language)**
* **Pivot → Target**

In this example, we demonstrate **Kannada → English → Hindi** translation using Facebook’s `M2M100` model.

---

### Why Pivot Translation?

Some languages (like Kannada or Tamil) may not have rich parallel corpora with every target language. Instead of training a direct model, we leverage a common intermediate language — typically English — which is well-supported and allows for more accurate two-step translation.

---

### Setup

We use `facebook/m2m100_418M`, a multilingual model trained on many language pairs without needing English as a bridge.





In [ ]:
!pip install transformers datasets torch accelerate --quiet

In [29]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Load model and tokenizer
model_name = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)

### 🔹 Step 1: Kannada → English

First, we translate the Kannada sentence `"ನಾನು ಈ ಪುಸ್ತಕವನ್ನು ಇಷ್ಟಪಟ್ಟೆನೆ"` (“I liked this book”) to English.

In [ ]:
kannada_text = "ನಾನು ಈ ಪುಸ್ತಕವನ್ನು ಇಷ್ಟಪಟ್ಟೆನೆ"
tokenizer.src_lang = "kn"
encoded_kn = tokenizer(kannada_text, return_tensors="pt")

english_tokens = model.generate(
    **encoded_kn,
    forced_bos_token_id=tokenizer.get_lang_id("en")
)
english_translation = tokenizer.decode(english_tokens[0], skip_special_tokens=True)
print("Step 1 - Kannada to English:", english_translation)

### 🔹 Step 2: English → Hindi

Now, the English translation is translated into Hindi using the same model.

In [ ]:
tokenizer.src_lang = "en"
encoded_en = tokenizer(english_translation, return_tensors="pt")

hindi_tokens = model.generate(
    **encoded_en,
    forced_bos_token_id=tokenizer.get_lang_id("hi")
)
hindi_translation = tokenizer.decode(hindi_tokens[0], skip_special_tokens=True)
print("Step 2 - English to Hindi:", hindi_translation)

### Summary

| Stage             | Input (Source)                   | Output (Target)                                   |
| ----------------- | -------------------------------- | ------------------------------------------------- |
| Kannada → English | "ನಾನು ಈ ಪುಸ್ತಕವನ್ನು ಇಷ್ಟಪಟ್ಟೆನೆ" | `"I liked this book"`                             |
| English → Hindi   | `"I liked this book"`            | Hindi translation (e.g., "आपकी आंखों की चपेट में") |

This pivoting approach can be especially useful for **zero-resource** or **low-resource** language pairs using a common multilingual translation backbone.